In [336]:

using LinearAlgebra
using GenericLinearAlgebra


In [337]:
function complete_orthonormal(U::Matrix{Float64})
    d, r = size(U)
    r ≤ d || error("U deve ter no máximo d colunas")

    r == d && return zeros(Float64, d, 0)   # já completa

    # Projeta vetores aleatórios no complemento de span(U)
    Z = randn(Float64, d, d - r)
    Z .-= U * (U' * Z)                      # remove componente em U

    # QR fino garante ortonormalidade
    C = Matrix(qr(Z).Q)                  # d × (d−r)
    return C
end

complete_orthonormal (generic function with 1 method)

In [338]:
function build_csd_input_matrix(D1::Matrix{Float64}, D2::Matrix{Float64})
    # Dimensões
    m, d = size(D1)  # D1: m x d
    n, _ = size(D2)  # D2: n x d

    # Passo 1: QR de D1^T para obter base de R(D1) em R^d
    Q1 = Matrix(qr(D1').Q)  # Q1 é d x m
    R1 = Matrix(qr(D1').R)  # R1 é m x m

    println("Q1: ", size(Q1))  # Debugging: verificar tamanho de Q1
    println("R1: ", size(R1))  # Debugging: verificar tamanho de R1

    r1 = sum(abs.(diag(R1)) .> eps() * maximum(abs.(diag(R1))))  # Estimar posto
    E1 = Q1[:, 1:r1]  
    E2 = complete_orthonormal(E1)  # Base ortonormal completa para R^d, d x (d - r1)
    E = hcat(E1, E2)  # Base ortonormal completa para R^d, d x d
    
    # Passo 2: QR de D2^T para obter base de R(D2) em R^d
    Q2 = Matrix(qr(D2').Q)  # Q2 é d x n
    R2 = Matrix(qr(D2').R)  # R2 é n x n

    println("Q2: ", size(Q2))  # Debugging: verificar tamanho de Q2
    println("R2: ", size(R2))  # Debugging: verificar tamanho de R2

    c1 = sum(abs.(diag(R2)) .> eps() * maximum(abs.(diag(R2))))  # Estimar posto
    F1 = Q2[:, 1:c1]  
    F2 = complete_orthonormal(F1)  # Base ortonormal completa para R^d, d x (d - c1)
    

    # Passo 3: Calcular Q11 = E1^T F1
    Q11 = E1' * F1  # r1 x c1
    Q12 = E1' * F2  # r1 x (d - c1)
    Q21 = E2' * F1  # (d - r1) x c1
    Q22 = E2' * F2  # (d - r1) x (d - c1)

    # Passo 4: Concatenar Q11, Q12, Q21, Q22
    Qtop = hcat(Q11, Q12)
    Qbot = hcat(Q21, Q22)
    Q = vcat(Qtop, Qbot)
    
    return Q, r1, c1, E, E1, F1, R1, R2
end

build_csd_input_matrix (generic function with 1 method)

In [339]:
# Exemplo com dados aleatórios
n = 100  # amostras dos datasets
d = 500  # número de features

D1 = randn(n, d)  # dataset 1
D2 = randn(n, d)  # dataset 2

Q, r1, c1, E, E1, F1, R1, R2 = build_csd_input_matrix(D1, D2)

Q1: (500, 100)
R1: (100, 100)
Q2: (500, 100)
R2: (100, 100)


([-0.05011037246017392 -0.03382373958515126 … 0.0071276254757617966 0.01892842319961343; -0.02199437066761421 -0.01760639363517051 … 0.030694847570907235 0.019094389385542407; … ; 0.08215977746321437 -0.020525020343034453 … -0.06233549817396504 -0.026025129666633347; 0.00031968117962985834 0.008151749463694364 … 0.029901544051093323 -0.03435009582907646], 100, 100, [-0.009895503410297657 0.03134405192226304 … 0.03128332557127645 -0.025387585887279303; -0.017196790226170326 -0.04730778856866724 … -0.08592083399870422 0.010366437996958397; … ; -0.02345386228572229 -0.016818062712386214 … -0.012865095452347484 -0.0705507472052162; -0.015420517673843757 -0.017043241357255764 … 0.07280756205144148 0.04206018400950789], [-0.009895503410297657 0.03134405192226304 … -0.024987473452282114 0.037173760396172165; -0.017196790226170326 -0.04730778856866724 … 0.012510029581356271 -0.029307753854120704; … ; -0.02345386228572229 -0.016818062712386214 … 0.019274985883682802 0.0981362773421699; -0.01542

In [340]:
display(size(Q))  # Verifica o tamanho da matriz Q
println("Posto de D1: ", r1)
println("Posto de D2: ", c1)
println("Tamanho de E: ", size(E))  # Verifica o tamanho da base ortonormal completa E
println("Tamanho de E1: ", size(E1))  # Verifica o tamanho de E1
println("Tamanho de F1: ", size(F1))  # Verifica o tamanho de E2
println("Tamanho de R1: ", size(R1))  # Verifica o tamanho de R1
println("Tamanho de R2: ", size(R2))  # Verifica o tamanho de R2

(500, 500)

Posto de D1: 100
Posto de D2: 100
Tamanho de E: (500, 500)
Tamanho de E1: (500, 100)
Tamanho de F1: (500, 100)
Tamanho de R1: (100, 100)
Tamanho de R2: (100, 100)


In [341]:
# Verificar se R1' * E1' é aproximadamente igual a D1
if R1'*E1' ≈ D1
    println("R1' * E1' é aproximadamente igual a D1")
else
    println("R1' * E1' é diferente de D1")
end

R1' * E1' é aproximadamente igual a D1


In [342]:
function rebuild_datasets_matrices(E, R)
    D_reconstructed = R' * E'  
    # Retornar a matriz reconstruída
    return D_reconstructed
end

rebuild_datasets_matrices (generic function with 1 method)

In [343]:
function verify_Q(Q::Matrix{Float64})
    if Q'*Q ≈ I
        println("A matriz Q é ortonormal")
    else
        println("A matriz Q não é ortonormal")
        
    end
end

verify_Q (generic function with 1 method)

In [344]:
# Exibir e verificar a corretude da matriz A
verify_Q(Q)
D1_reconstructed = rebuild_datasets_matrices(E1, R1)
D2_reconstructed = rebuild_datasets_matrices(F1, R2)

if D1_reconstructed ≈ D1 && D2_reconstructed ≈ D2
    println("As matrizes reconstruídas são equivalentes às originais!")
else
    println("As matrizes reconstruídas NÃO são equivalentes às originais.")
        
end

A matriz Q é ortonormal
As matrizes reconstruídas são equivalentes às originais!


In [345]:
function csd(Q, c1, r1, E)
    d= size(Q, 1)  # Dimensão do espaço
    # separar Q em blocos
    Q11 = Q[1:r1, 1:c1]  # r1 x c1
    Q12 = Q[1:r1, c1+1:end]  # r1 x (d - c1)
    Q21 = Q[r1+1:end, 1:c1]  # ( d - r1) x c1
    Q22 = Q[r1+1:end, c1+1:end]  # (d - r1) x (d - c1)

    # SVD dos blocos de Q = EᵀF
    U1, S11, V1 = svd(Q11; full=true)     
    U2, S22, V2  = svd(Q22; full=true)

    D11 = U1' * Q11 * V1
    D12 = U1' * Q12 * V2 
    D21 = U2' * Q21 * V1
    D22 = U2' * Q22 * V2
    
    # Monta matriz diagonalizada D via blocos projetados
    D_top = hcat(D11, D12)  # r1 x d
    D_bot = hcat(D21, D22)  # (d - r1) x (d - c1)
    D = vcat(D_top, D_bot)  # d × d

    # Constrói matriz U unificada
    U = Matrix{Float64}(I, d, d)
    U[1:r1, 1:r1] = U1
    U[r1+1:end, r1+1:end] = U2
    
    # Constrói matriz V unificada
    V= Matrix{Float64}(I, d, d)
    V[1:c1, 1:c1] = V1
    V[c1+1:end, c1+1:end] = V2
    
    # Calcula os ângulos θ entre os subespaços
    θ_rad = acos.(clamp.(S11, -1.0, 1.0))
    θ_deg = θ_rad .* (180 / π)

    # Reconstrói a rotação W que leva E1 → F1
    W = E * U * D * U' * E'


    return U, U1, U2, V, V1, V2, D, θ_deg, W  
end

csd (generic function with 2 methods)

In [346]:
U, U1, U2, V, V1, V2, D, θ, W =csd(Q, c1, r1, E)

([0.10208676082196175 0.03410152788589936 … 0.0 0.0; 0.13235600794951236 0.07174117855937553 … 0.0 0.0; … ; 0.0 0.0 … 0.002746516320802585 0.0342833576581628; 0.0 0.0 … 0.012374123272849468 -0.024870670441100567], [0.10208676082196175 0.03410152788589936 … 0.16505957940540783 0.06349279535281956; 0.13235600794951236 0.07174117855937553 … 0.062092948297010175 -0.0415084153955878; … ; -0.21786647338568405 -0.018091279818020965 … -0.022487325728762178 0.09192889052517922; 0.2023605785830218 -0.13871604846654953 … -0.019457907832094105 -0.20758946976259626], [0.10525053962884086 -0.08049685469065275 … -0.008469796297032229 0.06309735725201754; -0.023539170260942027 0.09166064274856553 … 0.011559896076486742 0.04493553829996524; … ; -0.04697026045482894 0.06588065023844794 … 0.002746516320802585 0.0342833576581628; -0.006206519136386514 0.01934809682358285 … 0.012374123272849468 -0.024870670441100567], [0.021657842849174047 -0.016564515017460004 … 0.0 0.0; 0.0298326272637828 0.1375771315352

In [347]:
println("Ângulos principais entre subespaços (em graus): ", θ)
println("tamanho de U: ", size(U))
println("tamanho de U1: ", size(U1))
println("tamanho de U2: ", size(U2))
println("tamanho de V: ", size(V))
println("tamanho de V1: ", size(V1))
println("tamanho de V2: ", size(V2))
println("tamanho de D: ", size(D))
println("tamanho de W: ", size(W))

Ângulos principais entre subespaços (em graus): [38.41226558473219, 40.81336913549106, 41.01518020431577, 42.13269775256886, 42.51695086676158, 43.69400693301058, 44.45824566927296, 44.99567588447286, 45.37200252805409, 46.089567168804024, 46.894106374518685, 47.25911585097456, 48.03464107427265, 48.56078857884162, 48.943960235219535, 49.890817794774286, 50.24751733215085, 50.686384366931605, 51.57176466582555, 52.289331492035004, 52.58823270257033, 52.686348584163326, 53.12964422283068, 54.378934829910285, 54.60440834544437, 54.98894310309453, 55.90415369115849, 55.98741902678928, 56.16742853820614, 56.92626114205597, 57.40961784208407, 58.20010247585518, 58.835645447329036, 59.05824060394099, 59.29316510294142, 59.85896374243768, 60.568050329638844, 60.928380234827074, 61.4848178363437, 62.354459586225396, 62.66982374097205, 63.15903492267742, 63.33580583428889, 63.85940588651102, 64.45564801283209, 65.46242996342595, 65.50227914695185, 66.13368154149187, 66.59864176390919, 67.352182

In [348]:
display(θ)                 # Ângulos principais entre subespaços (em radianos)
@show(U*D*V' ≈ Q) 
display(norm(W-I) ≈ norm(D-I))  # Deve ser próximo de 0 (rotação direta correta)
@show W*E1*U1*V1' ≈ F1 

100-element Vector{Float64}:
 38.41226558473219
 40.81336913549106
 41.01518020431577
 42.13269775256886
 42.51695086676158
 43.69400693301058
 44.45824566927296
 44.99567588447286
 45.37200252805409
 46.089567168804024
  ⋮
 85.80332979434046
 86.17770765058768
 86.89805067425696
 87.42281333460018
 88.18575576420606
 88.56301754907224
 89.21092789433564
 89.34656822708699
 89.64845696121589

true

U * D * V' ≈ Q = true
W * E1 * U1 * V1' ≈ F1 = true


true

In [352]:
D2_reconstructed_from_rotation = rebuild_datasets_matrices(W*E1*U1*V1', R2)

if  D2_reconstructed_from_rotation ≈ D2
    println("A matriz D2 reconstruída a partir da rotação é equivalente à original!")
else
    println("A matriz D2 reconstruída a partir da rotação NÃO é equivalente à original.")
end

A matriz D2 reconstruída a partir da rotação é equivalente à original!
